# Binary Classification | Binær klassifikation

Indtil videre har du kun oprettet regressionsmodeller. Det vil sige, at du har oprettet modeller, der producerede flydende punktprediktioner, såsom "huse i dette område koster N tusind dollars." I denne vejledning vil du oprette og evaluere en binær [klassifikationsmodel](https://developers.google.com/machine-learning/glossary/#classification_model). Det vil sige, at du vil oprette en model, der besvarer en binær spørgsmål. I dette øvelse vil spørgsmålet være, "Er huse i dette område over en bestemt pris?"

## Læringsmål

Efter at have gennemført denne vejledning, vil du kunne:

  * Konvertere et regressionsspørgsmål til et klassifikationsspørgsmål.
  * Ændre klassifikationsgrænsen og bestemme, hvordan denne ændring påvirker modellen.
  * Experimentere med forskellige klassifikationsmetrikker for at bestemme modellens effektivitet.

## Datasættet

Vores datasæt er Californien Housing Data. Som kan findes på: https://developers.google.com/machine-learning/crash-course/california-housing-data-description


## Indlæsning af nødvendige moduler

Følgende kode importerer de nødvendige moduler.

In [1]:

import numpy as np
import pandas as pd
import tensorflow as tf
from keras import layers
from matplotlib import pyplot as plt

# The following lines adjust the granularity of reporting.
pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.1f}".format
# tf.keras.backend.set_floatx('float32')

print("Ran the import statements.")

2025-02-19 11:09:11.739275: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739959751.771684   23091 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739959751.781941   23091 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-19 11:09:11.811740: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Ran the import statements.


## Hent datasættet fra internettet

Følgende kode celler loader de separate .csv filer og opretter følgende to pandas DataFrames:

* `train_df`, som indeholder træningssættet
* `test_df`, som indeholder test sættet


In [2]:
train_df = pd.read_csv("https://download.mlcc.google.com/mledu-datasets/california_housing_train.csv")
test_df = pd.read_csv("https://download.mlcc.google.com/mledu-datasets/california_housing_test.csv")
train_df = train_df.reindex(np.random.permutation(train_df.index)) # shuffle the training set

## Normalisering af datasættet

Når du opretter en model med flere funktioner, skal værdierne for hver funktion dække omtrent samme område. For eksempel, hvis et område har en værdi på 500 til 100.000 og et andet område har en værdi på 2 til 12, så vil modellen være svær at træne. Derfor skal du
[normalisere](https://developers.google.com/machine-learning/glossary/#normalization) funktioner i en model med flere funktioner.


Følgende kode normaliserer datasættene ved at konvertere hver rå værdi (inklusive labelen) til sin Z-score. En **Z-score** er antallet af standardafvigelser fra middelværdien for en bestemt rå værdi. For eksempel, overvej en funktion, der har følgende karakteristika:


  * Middelværdien er 60.
  * Standardafvigelsen er 10.


Den rå værdi 75 ville have en Z-score på +1.5:


```
  Z-score = (75 - 60) / 10 = +1.5
```

Den rå værdi 38 ville have en Z-score på -2.2:

```
  Z-score = (38 - 60) / 10 = -2.2
```

In [3]:
# Calculate the Z-scores of each column in the training set and
# write those Z-scores into a new pandas DataFrame named train_df_norm.
train_df_mean = train_df.mean()
train_df_std = train_df.std()
train_df_norm = (train_df - train_df_mean)/train_df_std

# Examine some of the values of the normalized training set. Notice that most
# Z-scores fall between -2 and +2.
train_df_norm.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
7217,0.6,-0.8,0.7,-0.9,-1.0,-0.9,-1.0,1.1,-0.4
235,1.5,-0.9,-1.2,2.8,3.2,-0.1,0.5,-0.1,-0.5
15731,-1.4,1.0,1.9,-0.5,-0.4,-0.1,-0.4,-0.5,0.3
8450,0.5,-0.7,-0.4,0.1,0.8,0.7,0.7,-0.6,-0.1
1847,1.1,-1.2,-1.7,0.5,0.5,0.2,0.4,-0.4,1.4


In [4]:
# Calculate the Z-scores of each column in the test set and
# write those Z-scores into a new pandas DataFrame named test_df_norm.
test_df_norm = (test_df - train_df_mean) / train_df_std

# Note that we transform the test data with the values calculated from the training set,
# as you should always transform your datasets with exactly the same values.

## Opgave 1: Opret en binær label

I klassifikationsproblemer skal labelen for hver eksempel være enten 0 eller 1. Desværre indeholder den naturlige label i Californien Housing Datasættet, `median_house_value`, floating-point værdier som 80.100 eller 85.700 i stedet for 0'er og 1'er, mens den normaliserede version af `median_house_values` indeholder floating-point værdier primært mellem -3 og +3.


Din opgave er at oprette en ny kolonne med navnet `median_house_value_is_high` i både træningssættet og testsættet. Hvis `median_house_value` er højere end en bestemt vilkårlig værdi (defineret af `threshold`), så sæt `median_house_value_is_high` til 1. Ellers sæt `median_house_value_is_high` til 0.


**Hint:** Cellerne i `median_house_value_is_high` kolonnen skal hver isætte `1` og `0`, ikke `True` og `False`. For at konvertere `True` og `False` til `1` og `0`, kalder du pandas DataFrame funktionen `astype(float)`.


In [7]:
threshold = 265000 # This is the 75th percentile for median house values.
train_df_norm["median_house_value_is_high"] = train_df_norm["median_house_value"].gt(threshold).astype(float)
test_df_norm["median_house_value_is_high"] = test_df_norm["median_house_value"].gt(threshold).astype(float)

# Print out a few example cells from the beginning and
# middle of the training set, just to make sure that
# your code created only 0s and 1s in the newly created
# median_house_value_is_high column
train_df_norm["median_house_value_is_high"].head(8000)

7217    0.0
235     0.0
15731   0.0
8450    0.0
1847    0.0
         ..
9372    0.0
8955    0.0
726     0.0
3859    0.0
15844   0.0
Name: median_house_value_is_high, Length: 8000, dtype: float64

## Reprensentere funktioner som inputlag

Denne kode celle specificerer funktionerne, `median_income` og ` total_rooms`, som du vil træne modellen på. Disse [Input](https://www.tensorflow.org/api_docs/python/tf/keras/Input) objekter er instansieret som Keras tensors.


In [8]:
inputs = {
# Features used to train the model on.
  'median_income': tf.keras.Input(shape=(1,)),
  'total_rooms': tf.keras.Input(shape=(1,))
}


## Definere funktioner, der bygger og træner en model

Denne kode celle definerer to funktioner: 

  * `create_model(inputs, learning_rate, METRICS)`, som definerer modellens
    topografi.

  * `train_model(model, dataset, epochs, label_name, batch_size, shuffle)`, bruger input funktioner og labels til at træne modellen.

I dette øvelseseksempel bruger vi [sigmoid](https://developers.google.com/machine-learning/glossary#sigmoid-function) som aktiveringsfunktion.


In [ ]:
def create_model(my_inputs, my_learning_rate, METRICS):
  # Use a Concatenate layer to concatenate the input layers into a single tensor.
  # as input for the Dense layer. Ex: [input_1[0][0], input_2[0][0]]
  concatenated_inputs = tf.keras.layers.Concatenate()(my_inputs.values())
  dense = layers.Dense(units=1, name='dense_layer', activation=tf.sigmoid)
  dense_output = dense(concatenated_inputs)
  """Create and compile a simple classification model."""
  my_outputs = {
    'dense': dense_output,
  }
  model = tf.keras.Model(inputs=my_inputs, outputs=my_outputs)

  # Call the compile method to construct the layers into a model that
  # TensorFlow can execute.  Notice that we're using a different loss
  # function for classification than for regression.
  model.compile(optimizer=tf.keras.optimizers.experimental.RMSprop(learning_rate=my_learning_rate),
                loss=tf.keras.losses.BinaryCrossentropy(),
                metrics=METRICS)
  return model


def train_model(model, dataset, epochs, label_name,
                batch_size=None, shuffle=True):
  """Feed a dataset into the model in order to train it."""

  # The x parameter of tf.keras.Model.fit can be a list of arrays, where
  # each array contains the data for one feature.  Here, we're passing
  # every column in the dataset. Note that the feature_layer will filter
  # away most of those columns, leaving only the desired columns and their
  # representations as features.
  features = {name:np.array(value) for name, value in dataset.items()}
  label = np.array(features.pop(label_name))
  history = model.fit(x=features, y=label, batch_size=batch_size,
                      epochs=epochs, shuffle=shuffle)

  # The list of epochs is stored separately from the rest of history.
  epochs = history.epoch

  # Isolate the classification metric for each epoch.
  hist = pd.DataFrame(history.history)

  return epochs, hist

print("Defined the create_model and train_model functions.")

Defined the create_model and train_model functions.


## Definere en plotting funktion

Denne [matplotlib](https://developers.google.com/machine-learning/glossary/#matplotlib) funktion plotter en eller flere kurver, der viser, hvordan forskellige klassifikationsmålinger ændrer sig med hver epoch.


In [10]:
def plot_curve(epochs, hist, list_of_metrics):
  """Plot a curve of one or more classification metrics vs. epoch."""
  # list_of_metrics should be one of the names shown in:
  # https://www.tensorflow.org/tutorials/structured_data/imbalanced_data#define_the_model_and_metrics

  plt.figure()
  plt.xlabel("Epoch")
  plt.ylabel("Value")

  for m in list_of_metrics:
    x = hist[m]
    plt.plot(epochs[1:], x[1:], label=m)

  plt.legend()

print("Defined the plot_curve function.")

Defined the plot_curve function.


## Kald funktionerne til at oprette og træne modellen, og derefter plotte resultaterne.

Denne kode celle specificerer hyperparametrene, og derefter kalder funktionerne til at oprette og træne modellen, og derefter plotte resultaterne.

In [ ]:
# The following variables are the hyperparameters.
learning_rate = 0.001
epochs = 20
batch_size = 100
label_name = "median_house_value_is_high"
classification_threshold = 0.35

# Establish the metrics the model will measure.
METRICS = [
           tf.keras.metrics.BinaryAccuracy(name='accuracy',
                                           threshold=classification_threshold),
          ]

# Establish the model's topography.
my_model = create_model(inputs, learning_rate, METRICS)

# To view a PNG of this model's layers, uncomment the call to
# `tf.keras.utils.plot_model` below. After running this code cell, click
# the file folder on the left, then the `my_classification_model.png` file.
# tf.keras.utils.plot_model(my_model, "my_classification_model.png")

# Train the model on the training set.
epochs, hist = train_model(my_model, train_df_norm, epochs,
                           label_name, batch_size)

# Plot a graph of the metric(s) vs. epochs.
list_of_metrics_to_plot = ['accuracy']

plot_curve(epochs, hist, list_of_metrics_to_plot)

ValueError: Only input tensors may be passed as positional arguments. The following argument value should be passed as a keyword argument: dict_values([<KerasTensor shape=(None, 1), dtype=float32, sparse=None, name=keras_tensor>, <KerasTensor shape=(None, 1), dtype=float32, sparse=None, name=keras_tensor_1>]) (of type <class 'dict_values'>)

Accuracy skal gradvist blive bedre under træningen (indtil det ikke kan forbedres yderligere).


## Evaluer modellen mod test sættet

Efter at have trænet modellen, fik du en vis præcision mod *træningssættet*. Kald følgende kode celle for at bestemme din modelens præcision mod *test sættet*.

In [ ]:
features = {name:np.array(value) for name, value in test_df_norm.items()}
label = np.array(features.pop(label_name))

my_model.evaluate(x = features, y = label, batch_size=batch_size)

## Opgave 2: Hvor præcis er din model?

Er din model værdifuld? Giv dit svar herunder:

## Opgave 3: Tilføj precision og recall som målinger

At bruge accuracy, isoleret set, kan være en dårlig måde at vurdere en klassifikationsmodel på. Modificer koden i følgende kode celle for at gøre modellen til at måle ikke kun accuracy, men også precision og recall. Vi har tilføjet accuracy og precision; din opgave er at tilføje recall. Se [TensorFlow Reference](https://www.tensorflow.org/api_docs/python/tf/keras/metrics/Recall) for detaljer.

In [ ]:
# The following variables are the hyperparameters.
learning_rate = 0.001
epochs = 20
batch_size = 100
classification_threshold = 0.35
label_name = "median_house_value_is_high"

# Modify the following definition of METRICS to generate
# not only accuracy and precision, but also recall:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy',
                                      threshold=classification_threshold),
      tf.keras.metrics.Precision(thresholds=classification_threshold,
                                 name='precision'
                                 ),
      ?  # write code here
]

# Establish the model's topography.
my_model = create_model(inputs, learning_rate, METRICS)

# Train the model on the training set.
epochs, hist = train_model(my_model, train_df_norm, epochs,
                           label_name, batch_size)

# Plot metrics vs. epochs
list_of_metrics_to_plot = ['accuracy', 'precision', 'recall']
plot_curve(epochs, hist, list_of_metrics_to_plot)

## Opgave 4: Eksperimenter med klassifikationsgrænsen (Hvis du har ekstra tid)

Eksperimenter med forskellige værdier for `classification_threshold` i koden i cellen "Invoke the creating, training, and plotting functions."  Hvilken værdi for `classification_threshold` producerer den højeste præcision?


## Opgave 5: Opsummere modellens ydeevne (Hvis du har ekstra tid)

Hvis du har ekstra tid, tilføj en anden måling, der forsøger at opsummere modellens samlede ydeevne.
